# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx

# Tutoriales N° 12 y 13: Segmentación y Clasificación de Imágenes.

En este ejercicio, desarrollaremos un ejemplo de segmentación y clasificación automatizada de imágenes multiespectrales, utilizando la plataforma Google Earth Engine (para selección y segmentación de imágenes), y `sklearn`(para clasificación). Este ejercicio es fácilmente extendible a otros tipos de imágenes multiespectrales, a otras regiones geográficas (para las cuales cuente con datos de entrenamiento), y a otros modelos de clasificación.

La zona de estudio corresponde a un área cultivada en California, EEUU. El objetivo, es detectar automáticamente distintos tipos de cultivos, de acuerdo a lo definido en el dataset etiquetado `labeled_points.gpkg`, que corresponde a observaciones registradas en el año 2020. 

Las clases de cultivos a identificar corresponden a las definiciones de USDA National Agricultural Statistics Service, disponibles en:

https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL

Este ejercicio será dividido en 2 partes:
* Parte I: Segmentación de Imágenes y Cálculo de Estadísticas Zonales (Tutorial N°12)
* Parte II: Clasificación y Predicción (Tutorial N°13)


In [ ]:
import ee,geemap
import geopandas as gpd
import pandas as pd

In [ ]:
#Esta ruta debe adecuarse a su entorno local
path='/Users/paulaaguirre/Downloads/'

In [ ]:
#ee.Authenticate()

In [ ]:
ee.Initialize()

El área de estudio está definida por los siguientes límites:

In [ ]:
aoi= ee.Geometry.Polygon([[[-121.89511299133301, 38.98496606984683],
                             [-121.89511299133301, 38.909335196675435],
                             [-121.69358253479004, 38.909335196675435],
                             [-121.69358253479004, 38.98496606984683]]]);

# Parte I: Segmentación de Imágenes (Tutorial N°12)

## 1. Selección de imagen Sentinel 2 para el año 2016.

Como insumo para la segmentación y clasificación, utilizaremos imágenes de Sentinel 2, para el año 2016. Obtenemos la mediana del año 2016 siguiendo el proceso habitual:

In [ ]:
def mask_s2_clouds(image):
  
  qa = image.select('QA60')

  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))

  return image.updateMask(mask).divide(10000)

In [ ]:
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2020-01-01', '2021-01-01').filterBounds(aoi).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(mask_s2_clouds))
dataset

In [ ]:
#Definimos una función para el cálculo de varios índices espectrales, a usar como features en la clasificación.

def addIndexes(image):    
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
  nbr = image.normalizedDifference(['B8', 'B12']).rename('nbr')
  ndwi=image.normalizedDifference(['B3', 'B8']).rename('ndwi')
  ndbi=image.normalizedDifference(['B11', 'B9']).rename('ndbi')
  ndmi=image.normalizedDifference(['B8', 'B11']).rename('ndmi')

  saviExp = '(b("B8")-b("B4"))/ (b("B8")+b("B4")+0.428)*1.428';
  savi = image.expression(saviExp).rename('SAVI');

    
  newBands = ee.Image([ndvi, nbr,ndwi,ndbi,ndmi,savi])
  image = image.addBands(newBands)
  return image

In [ ]:
#calculamos y recortamos la mediana del año. Mantenemos la escala de 10m, puede re-escalar (por ejemplo a 30 m) si es necesario.
dataset_indexes = dataset.map(addIndexes)

im = dataset_indexes.median().clip(aoi)

#Visualizamos la mediana RGB
vis = {'min': 0.0, 'max': 0.4,'bands': ['B4', 'B3', 'B2'],}

Map=geemap.Map()
Map.centerObject(aoi, 13)
Map.addLayer(im, vis, 'RGB', True)
Map

## 2. Segmentación con SNIC.

Para la segmentación, utilizaremos la implementación de SNIC en GEE:

https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic

In [ ]:
im

In [ ]:
#seleccionamos un conjunto de bandas para la segmentación 
bands=['B8','B4','B3','B2','ndvi','ndwi','ndbi','ndmi','SAVI']
ims=im.select(bands)

## NOTA: Se recomienda probar con distintos valores de los parámetros size, compactness y connectivity, y comparar los resultados obtenidos.

snic = ee.Algorithms.Image.Segmentation.SNIC(image=ims,size=50,compactness=0.1,connectivity=4);
snic

In [ ]:
#Capa raster con los segmentos

clusters = snic.select('clusters')
clusters

In [ ]:

#Vectorizamos el raster con los segmentos usando la función reduceToVectors() de EE, y obtenemos un FeatureCollection
fc = clusters.reduceToVectors(geometry=aoi,eightConnected=True,scale=10)
fc

In [ ]:
geemap.ee_export_image(clusters, filename='%s/clusters.tif'%path, region=aoi,scale=10,file_per_band=False)

In [ ]:
#Para analizar y visualizar los segmentos, exportamos el FeatureCollection a un geodataframe

gdf = geemap.ee_to_gdf(fc)
gdf=gdf.set_crs(crs=4326) #definir el crs 
gdf['oid']=gdf.index+1

gdf.head(3)

In [ ]:
len(gdf.label.unique())

In [ ]:
#Graficamos
gdf.plot(column='label',categorical=True,figsize=(12,10));

## 3. Cálculo de estadísticas zonales para cada cluster con GEE.

Para el entrenamiento de un modelo de clasificación, es necesario definir y calcular un conjunto de atributos o `features`de cada segmento. En este caso, utilizaremos los valores promedio, desviación estándar, valores mínimos y máximos de cada banda. Luego, para aplicar el clasificador sobre una nueva imagen, habrá que:
- Segmentar la nueva imagen (por ejemplo, con SNIC)
- Calcular las estadísticas para cada segmento, de manera de generar un dataframe con la misma estructura al utilizado para entrenar el clasificador
- Aplicar el modelo sobre este dataframe para generar predicciones para cada segmento.

In [ ]:
#Visualizemos nuevamente la imagen, y los segmentos
vis = {'min': 0.0,'max': 0.4,'bands': ['B4', 'B3', 'B2'],}

#mapa
Map=geemap.Map()
Map.centerObject(aoi, 13)
Map.addLayer(im, vis, 'RGB', True)
Map.addLayer(snic.randomVisualizer(), {}, 'clusters')

Map

- Para calcular las estadísticas zonales de cada segmento, podemos utilizar la función `zonal_statistics`de `geemap`: https://geemap.org/common/#geemap.common.zonal_stats

- Esta función requiere como inputs: la imagen sobre la cual calcular las estadísticas, y el conjunto de geometrías o `FeatureCollection`.

- Como output, escribe los resultados a un archivo en formato .csv (u otro formato a elección).

In [ ]:
allowed_statistics = {
        "COUNT"# ee.Reducer.count(),
        "MEAN"# ee.Reducer.mean(),
        "MEAN_UNWEIGHTED"# ee.Reducer.mean().unweighted(),
        "MAXIMUM"# ee.Reducer.max(),
        "MEDIAN"# ee.Reducer.median(),
        "MINIMUM"# ee.Reducer.min(),
        "MODE"# ee.Reducer.mode(),
        "STD"# ee.Reducer.stdDev(),
        "MIN_MAX"# ee.Reducer.minMax(),
        "SUM"# ee.Reducer.sum(),
        "VARIANCE"# ee.Reducer.variance(),
        "HIST"# ee.Reducer.histogram(maxBuckets=max_buckets, minBucketWidth=min_bucket_width, maxRaw=max_raw),
        "FIXED_HIST"# ee.Reducer.fixedHistogram(hist_min, hist_max, hist_steps),
        "COMBINED_COUNT_MEAN"# ee.Reducer.count().combine(ee.Reducer.mean(), sharedInputs=True),
        "COMBINED_COUNT_MEAN_UNWEIGHTED"# ee.Reducer.count().combine(ee.Reducer.mean().unweighted(), sharedInputs=True),
    }
    

Como estadística, podemos elegir una de las opciones anteriores, o combinar 2 estadísticas en un único objeto `Reducer` de GEE:
Para ello, es de ayuda la función `ee.Reducer.combine()`:

https://developers.google.com/earth-engine/apidocs/ee-reducer-combine

Para este ejemplo, combinaremos la media (mean) y desviación estándar (stdDev) de las bandas e índices espectrales seleccionados como features (toma un tiempo).

In [ ]:

reducers = ee.Reducer.mean().combine(reducer2=ee.Reducer.stdDev(),sharedInputs=True)

outfile='%s/stats.csv'%(path)

#Usamos la imagen con las bandas seleccionadas
stats=geemap.zonal_statistics(ims, fc,outfile, stat_type=reducers,scale=10)
df=pd.read_csv(outfile)
df

In [ ]:
df=pd.read_csv(outfile)
df

Recordemos el geodataframe creado a partir de la vectorización de segmentos:

In [ ]:
gdf[['geometry','label','oid']].columns

Combinamos este geodataframe, con los resultados de `zonal_statistics`:


In [ ]:
#Combinamos el geodataframe anterior, con los resultados de zonal_statistics
gdf=gdf[['geometry','label','oid']].merge(df,on='label',how='left')
#gdf.rename(columns={'B8':'B8_%s'%stat,'B4':'B4_%s'%stat,'B3':'B3_%s'%stat,'B2':'B2_%s'%stat,'B1':'B1_%s'%stat},inplace=True)
gdf.head()

In [ ]:
gdf.plot(column='ndvi_mean',figsize=(14,5),cmap='Greens',legend=True);

# Parte II: Clasificación y Predicción (Tutorial N°13)

## 4. Datos de entrenamiento.

In [ ]:
#training data
sample=gpd.read_file('labeled_points.gpkg',driver='GPKG')
sample.head()

In [ ]:
fig=plt.figure(figsize=(14,5))
ax=fig.add_subplot(111)
gdf.plot(ax=ax,column='label',categorical=True,alpha=0.5);
sample.plot(ax=ax,marker='.',color='k');


In [ ]:
#¿a qué cultivos corresponden?
cropcolors=pd.read_csv('crop_colors.csv',delimiter=';')
cropcolors

In [ ]:
sample=sample.merge(cropcolors,left_on='class',right_on='Value',how='left')
sample.head()

#### Leemos la imagen segmentada:

In [ ]:
import rasterio
#leemos la imagen de clusters
clusters = rasterio.open("%s/clusters.tif"%path)
clusters.count,clusters.height,clusters.width

In [ ]:
#agregamos a la lista el valor de cluster sobre el cual se ubica cada punto
coord_list = [(x,y) for x,y in zip(sample.lon ,sample.lat)]

#Evaluamos el raster de segmentación para cada punto etiquetado
dat=[x[0] for x in clusters.sample(coord_list)]
sample['cl_label']=dat
sample

In [ ]:
label_cl=gpd.sjoin(gdf,sample)
labeled_data=label_cl[['B8_mean', 'B8_stdDev', 'B4_mean',
       'B4_stdDev', 'B3_mean', 'B3_stdDev', 'B2_mean', 'B2_stdDev',
       'ndvi_mean', 'ndvi_stdDev', 'ndwi_mean', 'ndwi_stdDev', 'ndbi_mean',
       'ndbi_stdDev', 'ndmi_mean', 'ndmi_stdDev', 'SAVI_mean', 'SAVI_stdDev','class','Description']]
labeled_data

#### ¿Cómo se distribuyen los datos de entrenamiento por clases?

In [ ]:
import seaborn as sns

sns.catplot(data=labeled_data, x="class", kind="count");


## 5. Entrenamiento del modelo de clasificación con `sklearn`.


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
xcols=['B8_mean', 'B8_stdDev', 'B4_mean',
       'B4_stdDev', 'B3_mean', 'B3_stdDev', 'B2_mean', 'B2_stdDev',
       'ndvi_mean', 'ndvi_stdDev', 'ndwi_mean', 'ndwi_stdDev', 'ndbi_mean',
       'ndbi_stdDev', 'ndmi_mean', 'ndmi_stdDev', 'SAVI_mean', 'SAVI_stdDev']

X=labeled_data[xcols]
y=labeled_data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)


In [ ]:
clf = RandomForestClassifier()

# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, y_train)
y_pred_train = clf.predict(X_train)

print('Entrenamiento.')
print('Accuracy:',accuracy_score(y_train, y_pred_train))

# performing predictions on the test dataset
y_pred = clf.predict(X_test)


In [ ]:
y_pred

In [ ]:
from sklearn.metrics import *
print('Testeo.')

print('Precision:',precision_score(y_test, y_pred,average='weighted'))
print('Accuracy:',accuracy_score(y_test, y_pred))
print('F1-score:',f1_score(y_test, y_pred,average='weighted'))

Para un desarrollo más acabado del modelo, es posible refinar el entrenamiento del clasificador, por ejemplo probando con distintos métodos y ajustes de  hiperparámetros. Para mejorar su capacidad de generalización, también suele ser necesario mejorar la calidad y completitud de la data etiquetada.

## 6. Predicción para todos los segmentos.

In [ ]:
gdf[xcols]

In [ ]:
gdf['class_pred']=clf.predict(gdf[xcols])
gdf=gdf

In [ ]:
fig=plt.figure(figsize=(20,10))
ax=fig.add_subplot(111)

gdf.plot(ax=ax,column='class_pred',categorical=True,alpha=0.5,legend=True);
sample.plot(ax=ax,marker='.',color='k');


In [ ]:
cropcolors.head(10)

In [ ]:
gdf['class_pred'].value_counts()

In [ ]:
fig=plt.figure(figsize=(20,10))
ax=fig.add_subplot(111)

gdf.plot(ax=ax,column='class_pred',categorical=True,alpha=0.5,legend=True);
sample.plot(ax=ax,marker='.',color='k');


In [ ]:
gdf['class_pred']